In [1]:
import random
import copy

import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ThinkPad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ThinkPad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [3]:
quotes_frame = pd.read_csv('quotes_inspirational.csv', sep='\t')
print(quotes_frame.shape)
quotes = []
for q in quotes_frame['Quote']:
    quotes.append(nltk.word_tokenize(q))

(32949, 1)


In [4]:
def UNK(pos):
    return '<UNK_%s>' % pos

def isUNK(w):
    return w.startswith('<UNK_') and w.endswith('>')

In [5]:
N = 4
THRESHOLD = 10
TRAIN_SIZE = 10000

START = '<START>'
END = '<END>'

def gen_grams(quotes, N, threshold):
    quotes = copy.deepcopy(quotes)
    unk_words = {}
    word_count = {}
    # get word counts
    for q in quotes:
        for w in q:
            word_count[w] = word_count.get(w,0)+1
    # replace infrequent words with UNK
    for q in quotes:
        pos_tags = nltk.pos_tag(q)
        for i,w in enumerate(q):
            if word_count.get(w,0) < threshold:
                pos = UNK(pos_tags[i][1])
                if pos not in unk_words:
                    unk_words[pos] = []
                unk_words[pos].append(w)
                q[i] = pos
    # add padding to each sentence
    START_PAD = [START]*(N-1)
    END_PAD = [END]
    for x in range(len(quotes)):
        quotes[x] = START_PAD+quotes[x]+END_PAD
    # generate n-grams
    grams = {}
    for q in quotes:
        for i in range(len(q)-N+1):
            first = tuple(q[i:i+N-1])
            last = q[i+N-1]
            if first not in grams:
                grams[first] = []
            grams[first].append(last)
    return grams, unk_words

grams, unk_words = gen_grams(quotes[:TRAIN_SIZE], N, THRESHOLD)

In [6]:
def gen_sentence(N, grams, unk_words):
    # generate raw sentence with START, UNK, END
    ans = [START]*(N-1)
    while True:
        first = tuple(ans[-N+1:])
        nextword = random.choice(grams[first])
        ans.append(nextword)
        if nextword == END:
            break
    # get rid of START and END
    ans = ans[N-1:-1]
    # replace all UNK
    for i,w in enumerate(ans):
        if isUNK(w):
            ans[i] = random.choice(unk_words[w])
    return ' '.join(ans)


In [58]:
gen_sentence(N, grams, unk_words)

'To enjoy it again and again , until you do not need to eat animals , wear animals , or use animals for selfless miles , my dreams , I heard your overly akin , Smile keep you Human , Home keep you bottom !'